In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.utils import shuffle
warnings.filterwarnings('ignore')

Using TensorFlow backend.
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [2]:
# 모델 불러오기
model = joblib.load('model/model_mae_23.pkl')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
# test data의 데이터들 값 범위 파악 (min max)
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [4]:
# test의 행 별로 불러와서 각 열마다 -0.01~+0.01 사이 값을 더해줘서 데이터를 만들자
test_made = test.iloc[:,1:].copy()
for _ in range(40): # range 1 증가하면 data 만 개 늘어남
    test_made = np.append(test_made, test.iloc[:,1:] +np.random.uniform(low=-0.01,\
                                    high=0.01,size=test.iloc[:,1:].shape), axis=0)

test_made = pd.DataFrame(test_made)
test_made.columns = [str(x) for x in test_made.columns]

In [6]:
# model 4개로 각각 돌린 경우 이 부분을 수정해야 합니다!(합쳐야해요)
pred_layer = model.predict(test_made)

pred_layer = pd.DataFrame(pred_layer).rename(columns={0:'layer_1',
                                             1:'layer_2',
                                             2:'layer_3',
                                             3:'layer_4'})

(410000, 4)

In [8]:
made_df = pd.DataFrame(pd.concat([pred_layer, test_made], axis=1))

made_complete = pd.concat([train, made_df], axis=0)

# 데이터프레임의 row를 shuffle
made_complete = shuffle(made_complete)

# csv 파일로 저장 (시간 오래 걸림 주의!!)
made_complete.to_csv("data/model_mae_23_data.csv", header=True, index=False)